In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# === Parâmetros ===
ticker = "IVVB11.SA"
start_date = "2020-01-01"
end_date = "2024-12-31"
total_invested = 36000

# === Baixar dados e extrair preços ===
df_raw = yf.download(ticker, start=start_date, end=end_date)

# Selecionar coluna de preço com segurança
if isinstance(df_raw.columns, pd.MultiIndex):
    price_series = df_raw[('Close', ticker)].dropna()
else:
    price_series = df_raw['Adj Close'].dropna() if 'Adj Close' in df_raw else df_raw['Close'].dropna()

# Converter para DataFrame padrão
df = pd.DataFrame({'price': price_series})
df.index.name = 'Date'

# === Estratégias de datas ===
daily_dates = df.index
biweekly_dates = df.index[::10]
monthly_dates = df.groupby([df.index.year, df.index.month]).apply(lambda x: x.index[0])
monthly_dates = pd.DatetimeIndex(monthly_dates.values)

strategies = {
    'Diário': daily_dates,
    'Quinzenal': biweekly_dates,
    'Mensal': monthly_dates
}

# === Simulação ===
for name, dates in strategies.items():
    print(f"\n>>> Simulando {name}")
    invested = total_invested / len(dates)
    shares = 0.0
    dates_set = set(dates)

    df[f'{name}_shares'] = 0.0
    df[f'{name}_total'] = 0.0

    for date in df.index:
        if date in dates_set:
            price = df.loc[date, 'price']
            shares += invested / price
        df.loc[date, f'{name}_shares'] = shares
        df.loc[date, f'{name}_total'] = shares * df.loc[date, 'price']

    final_value = df[f'{name}_total'].iloc[-1]
    print(f"Valor final: R$ {final_value:,.2f}")

# === Plot ===
plt.figure(figsize=(12, 6))
for name in strategies:
    plt.plot(df.index, df[f'{name}_total'], label=name)
plt.title(f"Evolução da Carteira – {ticker}")
plt.ylabel("Valor da Carteira (R$)")
plt.xlabel("Data")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
